In [38]:
from load_blender import load_blender_data
class Args:
    def __init__(self):
        self.datadir = "./data/nerf_synthetic/lego"
        self.half_res = True
        self.testskip = 8
args = Args()
images, poses, render_poses, hwf, i_split = load_blender_data(args.datadir, args.half_res, args.testskip)
print('Loaded blender', images.shape, render_poses.shape, hwf, args.datadir)

Loaded blender (138, 400, 400, 4) torch.Size([40, 4, 4]) [400, 400, 555.5555155968841] ./data/nerf_synthetic/lego
